# Trading Code Generator

A code generator that writes trading code to buy and sell equities in a simulated environment, based on a given API


In [ ]:
# imports

import os
import sys
import io
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
import anthropic
import ollama
import gradio as gr
import requests
from typing import Any

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
client = genai.Client()


OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-sonnet-4-20250514"
GEMINI_MODEL = 'gemini-2.5-flash'
LLAMA_MODEL = "llama3.2"


In [ ]:
system_message = """
You are an effective programming assistant specialized to generate Python code based on the inputs.
Respond only with Python code; use comments sparingly and do not provide any explanation other than occasional comments.
Do not include Markdown formatting (```), language tags (python), or extra text.
"""

In [ ]:
def get_user_prompt_for_trade_code_generation(currency, wallet_balance):
    wallet_balance = str(wallet_balance)

    user_prompt = """
        Create a simple Crypto trading engine Python code.
        The engine will sell or buy the given crypto currency against USDT (Tether) based on the available wallet balance
        This should be a simple Python code, not a function
        The currency is: {}
        The wallet balance is: {}""".format(currency, wallet_balance)
    user_prompt += """
        Output will be a text containing the followings:
            - advice to sell or buy
            - amount in USDT
        Rules you have to apply in the code:
            - compose symbol: convert the input `crypto_currency` argument to upper case and concatenate it to string "USDT"
            - compose url passing the previously composed symbol: `url = f"https://data-api.binance.vision/api/v3/ticker/24hr?symbol={symbol}`
            - call the api from with this url, expect to get the following json response, for example:
                {'symbol': 'BTCUSDT',
                'priceChange': '1119.99000000',
                'priceChangePercent': '0.969',
                'weightedAvgPrice': '116314.23644195',
                'prevClosePrice': '115600.00000000',
                'lastPrice': '116719.99000000',
                'lastQty': '0.05368000',
                'bidPrice': '116719.99000000',
                'bidQty': '2.81169000',
                'askPrice': '116720.00000000',
                'askQty': '3.46980000',
                'openPrice': '115600.00000000',
                'highPrice': '117286.73000000',
                'lowPrice': '114737.11000000',
                'volume': '12500.51369000',
                'quoteVolume': '1453987704.98443060',
                'openTime': 1758015394001,
                'closeTime': 1758101794001,
                'firstId': 5236464586,
                'lastId': 5238628513,
                'count': 2163928}
            - build a logic based on the retrieving values which can decide whether the engine should sell or buy he given crypto currency
                - in the logic the code should also decide what is the confident level of selling or buying.
                    - if the confident level is high the `amount` should be higher (closer to the `current_wallet_balance`)
                    - if the confident level is lower then the amount should be lower as well
            - error handling:
                - if the api call returns with a json hving `code`, `msg` keys in it (eg. 'code': -1121, 'msg': 'Invalid symbol.') then handle this error message
        Response rule: in your response do not include Markdown formatting (```), language tags (python), or extra text.
    """
    return user_prompt


In [ ]:
def get_user_prompt_for_docstring_generation(python_code):
    return f"""
        Consider the following Python code: \n\n
        {python_code} \n\n

        Generate a docstring comment around this code and it alongside with the Python code. \n
        Response rule: in your response do not include Markdown formatting (```), language tags (python), or extra text.

        """

In [ ]:
def trade_gpt(currency, wallet_balance):
    completion = openai.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": get_user_prompt_for_trade_code_generation(
            currency,
            wallet_balance
            )
            }
    ]
    )
    return completion.choices[0].message.content

In [ ]:
def code_comment_gpt(python_code):
    completion = openai.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": get_user_prompt_for_docstring_generation(python_code)}
        ]
    )
    return completion.choices[0].message.content

In [ ]:
def trade_cloude(currency, wallet_balance):
    message = claude.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": get_user_prompt_for_trade_code_generation(
                currency,
                wallet_balance
                )
                },
        ],
    )

    return message.content[0].text

In [ ]:
def code_comment_cloude(python_code):
    message = claude.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": get_user_prompt_for_docstring_generation(python_code)
                },
        ],
    )

    return message.content[0].text

In [ ]:

def trade_gemini(currency, wallet_balance):
    response = client.models.generate_content(
    model=GEMINI_MODEL,
    config=types.GenerateContentConfig(
        system_instruction=system_message),
    contents=get_user_prompt_for_trade_code_generation(
        currency,
        wallet_balance
        )
    )

    return response.text


In [ ]:

def code_comment_gemini(python_code):
    response = client.models.generate_content(
    model=GEMINI_MODEL,
    config=types.GenerateContentConfig(
        system_instruction=system_message),
    contents=get_user_prompt_for_docstring_generation(python_code)
    )

    return response.text


In [ ]:
def trade_llama(currency, wallet_balance):
    completion = ollama.chat(
    model=LLAMA_MODEL,
    messages=[
                {"role": "user", "content": get_user_prompt_for_trade_code_generation(
                    currency,
                    wallet_balance
                    )
                    },
            ],
    )

    return completion['message']['content']


In [ ]:
def code_comment_llama(python_code):
    completion = ollama.chat(
    model=LLAMA_MODEL,
    messages=[
                {"role": "user", "content": get_user_prompt_for_docstring_generation(python_code)},
            ],
    )

    return completion['message']['content']

In [ ]:
def generate_python_code(input_model, currency, wallet_balance):
    model_mapping = {"GPT": trade_gpt(currency, wallet_balance),
                     "Claude": trade_cloude(currency, wallet_balance),
                     "Gemini": trade_gemini(currency, wallet_balance),
                     "Llama": trade_llama(currency, wallet_balance)}
    try:
        return model_mapping[input_model]
    except KeyError as e:
        print(f"{e}: {input_model} is not valid")


In [ ]:
def comment_python_code(input_model, python_code):
    model_mapping = {"GPT": code_comment_gpt(python_code),
                     "Claude": code_comment_cloude(python_code),
                     "Gemini": code_comment_gemini(python_code),
                     "Llama": code_comment_llama(python_code)
                     }
    try:
        return model_mapping[input_model]
    except KeyError as e:
        print(f"{e}: {input_model} is not valid")

In [ ]:
def execute_python(code):
    code = code.replace('```python\n','').replace('```','')
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("""It generate Trading Python code, which will recommend you whether sell or buy a given crypto currency at its current price.
                    Based on the confindency level of the prediction it will recommend what amount should be placed from your available wallet balance""")
    with gr.Row():
        crypto_currency = gr.Dropdown(["BTC", "ETH", "SOL"], label="The Crypto cyrrency")
        wallet_balance = gr.Number(label="Enter a number")
        model = gr.Dropdown(["GPT", "Claude", "Gemini", "Llama"], label="Select model", value="GPT")
    with gr.Row():
        generate_python_code_bt = gr.Button("Genarate Python code")
    with gr.Row():
        with gr.Column():
            python = gr.TextArea(label="Python Code")
            python_comment = gr.Button("Comment Python code")
            python_run = gr.Button("Run Python code")
    with gr.Row():
        result_out = gr.TextArea(label="Trading advice")

    generate_python_code_bt.click(generate_python_code, inputs=[model, crypto_currency, wallet_balance], outputs=[python])
    python_comment.click(comment_python_code, inputs=[model, python], outputs=python)
    python_run.click(execute_python, inputs=[python], outputs=result_out)

ui.launch(inbrowser=True)